In [55]:
import pandas as pd
import seaborn as sns
import re
import plotly.graph_objects as go
import plotly
import numpy as np

state_code = pd.read_csv('state-name-code.csv')
state_code.columns = [x.lower() for x in state_code.columns]
raw_data = pd.read_csv('all-state-changes.csv')
raw_data.shape

(3046, 20)

In [56]:
state_code.head()

,state,abbrev,code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [57]:
data = raw_data.copy()

# Cleaning State column, split electoral votes, convert to datetime
ev = []
stateCorrect = []
for i in range(len(data['state'])):
    temp = []
    st = ''
    for char in re.split(' |(|)',data['state'][i]):
        if (str(char).isdigit()):
            temp.append(char)
    ev.append(int(''.join(temp).strip()))
    op = data['state'][i].find('(')
    stateCorrect.append(data['state'][i][:op-1])
data['electoral_votes'] = ev
data['state'] = stateCorrect
data['timestamp'] = pd.DatetimeIndex(pd.to_datetime(data['timestamp'])).to_period('D')
data.head(1)

,state,timestamp,leading_candidate_name,trailing_candidate_name,leading_candidate_votes,trailing_candidate_votes,vote_differential,votes_remaining,new_votes,new_votes_relevant,...,leading_candidate_partition,trailing_candidate_partition,precincts_reporting,precincts_total,hurdle,hurdle_change,hurdle_mov_avg,counties_partition,total_votes_count,electoral_votes
0,Alaska,2020-11-19,Trump,Biden,189457,153551,35906,9210,7323,6977,...,0.528594,0.471406,407,441,2.527651,0.910664,0.489933,"{'ED 1': 164, 'ED 10': 312, 'ED 11': 174, 'ED ...",356796,3


In [61]:
# Daily reporting of leading candidate votes
years = []
for i in data['timestamp'].unique():
    temp = data[data['timestamp'] == i]
    aggre = temp.groupby('state').agg({'leading_candidate_votes':'max','trailing_candidate_votes':'max'})
    aggre = aggre.merge(state_code, on='state').reset_index(drop=True)
    tdata = {'type': 'choropleth',
             'colorscale': 'Greens',
             'locations': aggre['code'].astype(str),
             'z': aggre['leading_candidate_votes'].astype(float),
             'locationmode': 'USA-states'}
    years.append(tdata)
    
steps = []
for i in range(len(years)):
    step = {'method': 'restyle',
            'args': ['visible', [False] * len(years)],
            'label': f'Day {i}'}
    step['args'][1][i] = True
    steps.append(step)

sliders = [{'active': 0,
            'pad': {"t": 1},
            'steps': steps}]    

layout = dict(geo={'scope': 'usa',
                   'projection': {'type': 'albers usa'}},
              sliders=sliders)

fig_three = go.Figure(data = years, layout = layout)
fig_three.show()